In [24]:
#CAUTION: Takes a long time to load.
install.packages("themis")
devtools::install_github("tidymodels/tune")
install.packages("kknn")

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Skipping install of 'tune' from a github remote, the SHA1 (597ddf7a) has not changed since last install.
  Use `force = TRUE` to force installation

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

also installing the dependency ‘igraph’




In [2]:
#load libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(themis)
options(repr.matrix.max.rows = 10)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.4          ✔ rsample      1.1.1     
✔ dials        1.2.0          ✔ tune         1.1.1.9000
✔ infer        1.0.4          ✔ workflows    1.1.3     
✔ modeldata    1.1.0          ✔ workflowsets 1.0.1     
✔ parsnip      1.1.0          ✔ yardstick    1.1.0     
✔ recipes   

In [9]:
#Add columns
unscaled_data <- read_csv("pulsar_data.csv", 
                        col_names = c("mean_integrated_profile", 
                                      "stand_dev_integrated_profile", 
                                      "exc_kurtosis_integrated_profile", 
                                      "skew_integrated_profile",
                                      "mean_dmsnr", 
                                      "stand_dev_dmsnr", 
                                      "exc_kurtosis_dmsnr", 
                                      "skew_dmsnr", "class")) 

unscaled_data <- unscaled_data |>
    mutate(class = as_factor(class))
#unscaled_data 

#Class proportions in unscaled dataset (Imbalanced)
num_obs <- nrow(unscaled_data)
pulsar_proportions <- unscaled_data |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(unscaled_data))
pulsar_proportions

#Scale data
#pulsar_recipe <- recipe(class ~ exc_kurtosis_integrated_profile + skew_dmsnr + stand_dev_integrated_profile + 
#                        mean_dmsnr + skew_integrated_profile + exc_kurtosis_dmsnr, data = unscaled_data) |>
#    step_scale(all_predictors()) |>
#    step_center(all_predictors()) |>
#    prep()

#standardized_pulsar <- bake(pulsar_recipe, unscaled_data)
#standardized_pulsar

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): mean_integrated_profile, stand_dev_integrated_profile, exc_kurtosis...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


class,n,percent
<fct>,<int>,<dbl>
0,16259,90.842552
1,1639,9.157448


exc_kurtosis_integrated_profile,skew_dmsnr,stand_dev_integrated_profile,mean_dmsnr,skew_integrated_profile,exc_kurtosis_dmsnr,class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
-0.66955083,-0.2874301,1.3347944,-0.3194313,-0.4004478,-0.07279575,0
-0.01178443,0.2115755,1.8022150,-0.3710915,-0.3705251,0.50441285,0
-0.14522850,-0.3913625,-1.0532928,-0.3220980,-0.1165896,-0.12599257,0
-0.51339427,-0.4812869,1.5532110,-0.3043957,-0.3901672,-0.31225666,0
0.11560548,1.3867553,-0.8588548,-0.3879995,-0.1048632,1.32398915,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮
-0.6256372,1.6999868,1.9432298,-0.3839995,-0.4066857,1.586009,0
-0.3288215,-0.9065487,0.4290505,0.1287725,-0.2346366,-1.189126,0
-0.2993254,-0.9413038,1.9561650,0.2991291,-0.4074805,-1.288043,0


In [30]:
set.seed(1)
pulsar_split <- initial_split(unscaled_data, prop = 0.75, strata = class)
pulsar_train <- training(pulsar_split)
pulsar_test <- testing(pulsar_split) 

#Check class proportions in the training set
pulsar_proportions <- pulsar_train |>
    group_by(class) |>
    summarize(n = n()) |>
    mutate(percent = 100*n/nrow(pulsar_train)) 
pulsar_proportions

#Upscale data to balance training set
pulsar_recipe <- recipe(class ~ ., data = pulsar_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) |>
    themis::step_upsample(class, over_ratio = 1, skip = FALSE) |>
    prep(pulsar_train) 
pulsar_recipe

#Check new proportions in training set
#new_pulsar_proportions <- pulsar_recipe |>
#    group_by(class) |>
#    summarize(n = n())
#new_pulsar_proportions

dummy_recipe <- recipe(class ~ ., data = pulsar_recipe)

#Check new proportions in training set
#new_pulsar_proportions <- upsampled_train |>
#    group_by(class) |>
#    summarize(n = n()) |>
#    mutate(percent = 100*n/nrow(pulsar_upsampled))
#new_pulsar_proportions

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
    set_engine("kknn") |>
    set_mode("classification")

pulsar_fit <- workflow() |>
    add_recipe(pulsar_recipe) |>
    add_model(knn_spec) |>
    fit(data = pulsar_upsampled)
pulsar_fit

pulsar_test_predictions <- predict(pulsar_fit, pulsar_test) |>
    bind_cols(pulsar_test)
pulsar_test_predictions

pulsar_prediction_accuracy <- pulsar_test_predictions |>
    metrics(truth = class, estimate = .pred_class) |>
    filter(.metric == "accuracy")
pulsar_prediction_accuracy

confusion <- pulsar_test_predictions |>
    conf_mat(truth = class, estimate = .pred_class)
confusion

class,n,percent
<fct>,<int>,<dbl>
0,12200,90.888773
1,1223,9.111227




── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 8



── Training information 

Training data contained 13423 data points and no incomplete rows.



── Operations 

• Scaling for: mean_integrated_profile, ... | Trained

• Centering for: mean_integrated_profile, ... | Trained

• Up-sampling based on: class | Trained



ERROR: Error in as.data.frame.default(value, stringsAsFactors = FALSE): cannot coerce class ‘"recipe"’ to a data.frame


In [13]:
#10 fold cross validation
pulsar_vfold <- vfold_cv(pulsar_train, v = 10, strata = class)

pulsar_resample_fit <- workflow() |>
       add_recipe(pulsar_recipe) |>
       add_model(knn_spec) |>
       fit_resamples(resamples = pulsar_vfold)

pulsar_metrics <- collect_metrics(pulsar_resample_fit)
pulsar_metrics

ERROR: Error in eval_select_impl(NULL, .vars, expr(c(!!!dots)), include = .include, : object 'pulsar_train' not found


In [ ]:
#CAUTION: Takes a long time to load.
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")
#knn_tune

k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

knn_results <- workflow() |>
               add_recipe(pulsar_recipe) |>
               add_model(knn_tune) |>
               tune_grid(resamples = pulsar_vfold, grid = k_vals) |>
               collect_metrics()
#knn_results

accuracies <- knn_results |> 
       filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate")  



accuracy_versus_k


**Selecting Predictors**

In [ ]:
names <- colnames(pulsar_train |> select(-class))
names

In [ ]:
example_formula <- paste("class", "~", paste(names, collapse="+"))
example_formula

In [ ]:
# create an empty tibble to store the results
predictor_accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

In [ ]:
# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("class", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        pulsar_recipe <- recipe(as.formula(model_string), 
                                data = pulsar_train) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(pulsar_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = pulsar_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
predictor_accuracies <- predictor_accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
predictor_accuracies

Based on the data above, the highest accuracy is obtained by having exc_kurtosis_integrated_profile+skew_dmsnr+stand_dev_integrated_profile+mean_dmsnr as predictors